# CRO Project data mining / exploration of the delivery time, geolocation and sales value per seller

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import chisquare
from datetime import datetime

# Hide deprecated warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Investigate how to replace missing values for each case and complete the data set
def replace_missing_values_by_mean(data):
    return data.apply(lambda x: x.fillna(x.mean()), axis=0)

In [3]:
sellers_data = pd.read_csv('olist_sellers_dataset.csv')
orders_data = pd.read_csv('olist_orders_dataset.csv')
items_data = pd.read_csv('olist_order_items_dataset.csv')

In [4]:
sellers_data.shape

(3095, 4)

In [5]:
orders_data.shape

(99441, 8)

## Merge datasets

In [6]:
sellers_items = pd.merge(sellers_data, items_data, on='seller_id', how='left')

In [7]:
delivery_df = pd.merge(sellers_items, orders_data, on='order_id', how='left')

In [9]:
delivery_df.head(2)

,seller_id,seller_zip_code_prefix,seller_city,seller_state,order_id,order_item_id,product_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,4a90af3e85dd563884e2afeab1091394,1,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.2,9.56,9d6837f9700a3441e7183bff3bc4eef0,delivered,2017-08-21 20:35:44,2017-08-21 20:50:19,2017-08-29 20:33:29,2017-08-30 16:07:13,2017-09-01 00:00:00
1,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,6d953888a914b67350d5bc4d48f2acab,1,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.7,15.92,a973c4e3ad82777add3fa188f91dacea,delivered,2017-05-05 16:12:29,2017-05-05 16:25:11,2017-05-12 05:43:55,2017-06-02 16:57:44,2017-05-30 00:00:00


In [10]:
delivery_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 17 columns):
seller_id                        112650 non-null object
seller_zip_code_prefix           112650 non-null int64
seller_city                      112650 non-null object
seller_state                     112650 non-null object
order_id                         112650 non-null object
order_item_id                    112650 non-null int64
product_id                       112650 non-null object
shipping_limit_date              112650 non-null object
price                            112650 non-null float64
freight_value                    112650 non-null float64
customer_id                      112650 non-null object
order_status                     112650 non-null object
order_purchase_timestamp         112650 non-null object
order_approved_at                112635 non-null object
order_delivered_carrier_date     111456 non-null object
order_delivered_customer_date    110196 non-null 

### Drop unneeded columns

In [11]:
delivery_clean = delivery_df.drop(['seller_zip_code_prefix','order_status','order_purchase_timestamp',
                                  'order_delivered_carrier_date','order_estimated_delivery_date'], axis = 1) 

In [12]:
delivery_clean.shape

(112650, 12)

In [14]:
delivery_clean.head(2)

,seller_id,seller_city,seller_state,order_id,order_item_id,product_id,shipping_limit_date,price,freight_value,customer_id,order_approved_at,order_delivered_customer_date
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,4a90af3e85dd563884e2afeab1091394,1,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.2,9.56,9d6837f9700a3441e7183bff3bc4eef0,2017-08-21 20:50:19,2017-08-30 16:07:13
1,3442f8959a84dea7ee197c632cb2df15,campinas,SP,6d953888a914b67350d5bc4d48f2acab,1,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.7,15.92,a973c4e3ad82777add3fa188f91dacea,2017-05-05 16:25:11,2017-06-02 16:57:44


In [15]:
delivery_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 12 columns):
seller_id                        112650 non-null object
seller_city                      112650 non-null object
seller_state                     112650 non-null object
order_id                         112650 non-null object
order_item_id                    112650 non-null int64
product_id                       112650 non-null object
shipping_limit_date              112650 non-null object
price                            112650 non-null float64
freight_value                    112650 non-null float64
customer_id                      112650 non-null object
order_approved_at                112635 non-null object
order_delivered_customer_date    110196 non-null object
dtypes: float64(2), int64(1), object(9)
memory usage: 11.2+ MB


In [16]:
delivery_clean['order_approved_at'] = pd.to_datetime(delivery_clean['order_approved_at'],format='%Y-%m-%d',utc=False)


In [17]:
delivery_clean['order_delivered_customer_date'] = pd.to_datetime(delivery_clean['order_delivered_customer_date'],format='%Y-%m-%d',utc=False)


In [18]:
delivery_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 12 columns):
seller_id                        112650 non-null object
seller_city                      112650 non-null object
seller_state                     112650 non-null object
order_id                         112650 non-null object
order_item_id                    112650 non-null int64
product_id                       112650 non-null object
shipping_limit_date              112650 non-null object
price                            112650 non-null float64
freight_value                    112650 non-null float64
customer_id                      112650 non-null object
order_approved_at                112635 non-null datetime64[ns]
order_delivered_customer_date    110196 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(7)
memory usage: 11.2+ MB


### Calculate the shipping days per seller

In [19]:
shipping_time = (delivery_clean['order_delivered_customer_date'] - delivery_clean['order_approved_at']).dt.days
delivery_clean['shipping_time'] = shipping_time.values

In [20]:
#Transform all values from shipping time column to absolute values
delivery_clean['shipping_time'] = delivery_clean['shipping_time'].abs()

In [21]:
delivery_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 13 columns):
seller_id                        112650 non-null object
seller_city                      112650 non-null object
seller_state                     112650 non-null object
order_id                         112650 non-null object
order_item_id                    112650 non-null int64
product_id                       112650 non-null object
shipping_limit_date              112650 non-null object
price                            112650 non-null float64
freight_value                    112650 non-null float64
customer_id                      112650 non-null object
order_approved_at                112635 non-null datetime64[ns]
order_delivered_customer_date    110196 non-null datetime64[ns]
shipping_time                    110181 non-null float64
dtypes: datetime64[ns](2), float64(3), int64(1), object(7)
memory usage: 12.0+ MB


In [22]:
delivery_clean['shipping_time'].describe()

count    110181.000000
mean         11.548997
std           9.412955
min           0.000000
25%           6.000000
50%           9.000000
75%          15.000000
max         208.000000
Name: shipping_time, dtype: float64

## Mean, max, min of delivery timing/ seller

#### Mean of delivery per seller

In [23]:
delivery_mean_seller = delivery_clean[['order_id','seller_id','shipping_time']].drop_duplicates().groupby('seller_id').mean().reset_index()#.sort_values(by='seller_id')

In [24]:
#check the nulls
delivery_mean_seller.isnull().sum()

seller_id          0
shipping_time    125
dtype: int64

In [25]:
len(delivery_mean_seller)

3095

In [26]:
#Defitinition of how to replace the NaN values for the column shipping_time
bymean = ['shipping_time']

delivery_clean[bymean] = replace_missing_values_by_mean(delivery_clean[bymean])

In [27]:
#Rerun it to apply the replacement of NaNs.
delivery_mean_seller = delivery_clean[['order_id','seller_id','shipping_time']].drop_duplicates().groupby('seller_id').mean().reset_index()#.sort_values(by='seller_id')

In [28]:
#Recheck there is no nulls
delivery_mean_seller.isnull().sum()

seller_id        0
shipping_time    0
dtype: int64

In [29]:
#Rename column for Mean SHIPPING TIME
delivery_mean_seller = delivery_mean_seller.rename(columns={"shipping_time": "mean_shipping_time"})
len(delivery_mean_seller)

3095

#### Max of delivery per seller

In [30]:
delivery_max_seller = delivery_clean[['order_id','seller_id','shipping_time']].drop_duplicates().groupby('seller_id').max().reset_index()#.sort_values(by='seller_id')
#Rename column for MAX SHIPPING TIME
delivery_max_seller = delivery_max_seller.rename(columns={"shipping_time": "max_shipping_time"})
len(delivery_max_seller)
a = pd.merge(delivery_mean_seller, delivery_max_seller, on='seller_id', how='outer')


#### Min of delivery per seller

In [31]:
delivery_min_seller = delivery_clean[['order_id','seller_id','shipping_time']].drop_duplicates().groupby('seller_id').min().reset_index()#.sort_values(by='seller_id')
#Rename column for MIN SHIPPING TIME
delivery_min_seller = delivery_min_seller.rename(columns={"shipping_time": "min_shipping_time"})
b = pd.merge(a, delivery_min_seller, on='seller_id', how='left')


## Mean, max, min of freight value

#### Freight value mean per seller

In [32]:
freight_value_mean = delivery_clean[['order_id','seller_id','freight_value']].drop_duplicates().groupby('seller_id').mean().reset_index()#.sort_values(by='seller_id')
#Rename column for MEAN FREIGHT VALUE
freight_value_mean = freight_value_mean.rename(columns={"freight_value": "mean_freight_value"})
c = pd.merge(b, freight_value_mean, on='seller_id', how='left')


#### Freight value max per seller

In [33]:
freight_value_max = delivery_clean[['order_id','seller_id','freight_value']].drop_duplicates().groupby('seller_id').max().reset_index()#.sort_values(by='seller_id')
#Rename column for MAX FREIGHT VALUE
freight_value_max = freight_value_max.rename(columns={"freight_value": "max_freight_value"})
d = pd.merge(c, freight_value_max, on='seller_id', how='left')


#### Freight values min per seller

In [34]:
freight_value_min = delivery_clean[['order_id','seller_id','freight_value']].drop_duplicates().groupby('seller_id').min().reset_index()#.sort_values(by='seller_id')
#Rename column for MIN FREIGHT VALUE
freight_value_min = freight_value_min.rename(columns={"freight_value": "min_freight_value"})
e = pd.merge(d, freight_value_min, on='seller_id', how='left')


## Geolocation per seller

#### City per seller

In [35]:
city_seller = delivery_clean[['seller_id','seller_city']].drop_duplicates()#.groupby('seller_id').min().reset_index()#.sort_values(by='seller_city')
f = pd.merge(e, city_seller, on='seller_id', how='left')

In [36]:
len(delivery_clean['seller_id'].unique())

3095

#### State per seller

In [37]:
state_seller = delivery_clean[['seller_id','seller_state']].drop_duplicates()#.groupby('seller_id').min().reset_index()#.sort_values(by='seller_city')
g = pd.merge(f, state_seller, on='seller_id', how='left')


## Sales value per seller

In [38]:
#Calculate the value of items and add it as a new column
delivery_clean['items_value'] = items_value = (delivery_clean['order_item_id'])*(delivery_clean['price'])

In [39]:
#Calculate the total sales value per seller 
sales_value_seller = delivery_clean[['seller_id','items_value']].groupby('seller_id').sum().reset_index()#.sort_values(by='seller_city')

h = pd.merge(g, sales_value_seller, on='seller_id', how='left')


In [40]:
delivery_clean.head()

,seller_id,seller_city,seller_state,order_id,order_item_id,product_id,shipping_limit_date,price,freight_value,customer_id,order_approved_at,order_delivered_customer_date,shipping_time,items_value
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,4a90af3e85dd563884e2afeab1091394,1,ffb64e34a37740dafb6c88f1abd1fa61,2017-08-25 20:50:19,106.20,9.56,9d6837f9700a3441e7183bff3bc4eef0,2017-08-21 20:50:19,2017-08-30 16:07:13,8.0,106.20
1,3442f8959a84dea7ee197c632cb2df15,campinas,SP,6d953888a914b67350d5bc4d48f2acab,1,f4621f8ad6f54a2e3c408884068be46d,2017-05-11 16:25:11,101.70,15.92,a973c4e3ad82777add3fa188f91dacea,2017-05-05 16:25:11,2017-06-02 16:57:44,28.0,101.70
2,3442f8959a84dea7ee197c632cb2df15,campinas,SP,bc8a5de6abf5b14f98a6135a7fb46731,2,325a06bcce0da45b7f4ecf2797dd40e4,2017-09-05 12:50:19,10.80,2.42,1554ffe702931a062b4383b109accf63,2017-08-30 12:50:19,2017-09-01 16:51:26,2.0,21.60
3,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,035201c3c82a97f8a25dd6bd5244b2d5,1,1c36a5285f7f3b1ed2637d7c528ce5ff,2017-11-22 04:30:29,89.99,45.09,9facbfd2dd51a45404d58154b12ed2dd,2017-11-14 04:31:07,2017-11-21 23:26:35,7.0,89.99
4,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0504447548229e075dea8441b37b1e2a,1,8852eb03b04ec3268a66e9b696e25f6f,2017-10-06 02:14:42,199.90,21.89,4e2c1f15de98416a90c2ee06b55ccc9b,2017-09-30 02:14:42,2017-10-06 17:03:59,6.0,199.90


#### Sales value mean, max, min, median

In [41]:
sales_value_seller_mean = delivery_clean[['seller_id','items_value']].groupby('seller_id').mean().reset_index()#.sort_values(by='seller_city')
#Rename column for mean sales value
sales_value_seller_mean = sales_value_seller_mean.rename(columns={"items_value": "mean_sales_value"})

i = pd.merge(h, sales_value_seller_mean, on='seller_id', how='left')


In [42]:
sales_value_seller_max = delivery_clean[['seller_id','items_value']].groupby('seller_id').max().reset_index()#.sort_values(by='seller_city')
#Rename column for max sales value
sales_value_seller_max = sales_value_seller_max.rename(columns={"items_value": "max_sales_value"})

j = pd.merge(i, sales_value_seller_max, on='seller_id', how='left')


In [43]:
sales_values_seller_min = delivery_clean[['seller_id','items_value']].groupby('seller_id').min().reset_index()#.sort_values(by='seller_city')
#Rename column for min sales value
sales_values_seller_min = sales_values_seller_min.rename(columns={"items_value": "min_sales_value"})

k = pd.merge(j, sales_values_seller_min, on='seller_id', how='left')


In [45]:
sales_values_seller_median = delivery_clean[['seller_id','items_value']].groupby('seller_id').median().reset_index()#.sort_values(by='seller_city')
#Rename column for median sales value
sales_values_seller_median = sales_values_seller_median.rename(columns={"items_value": "median_sales_value"})
l_df = pd.merge(k, sales_values_seller_median, on='seller_id', how='left')


In [46]:
delivery_sales = l_df

In [47]:
delivery_sales

,seller_id,mean_shipping_time,order_id_x,max_shipping_time,order_id_y,min_shipping_time,mean_freight_value,order_id_x,max_freight_value,order_id_y,min_freight_value,seller_city,seller_state,items_value,mean_sales_value,max_sales_value,min_sales_value,median_sales_value
0,0015a82c2db000af6aaaf3ae2ecb0532,9.666667,d455a8cb295653b55abda06d434ab492,12.000000,7f39ba4c9052be115350065d07583cac,8.000000,21.020000,d455a8cb295653b55abda06d434ab492,21.02,7f39ba4c9052be115350065d07583cac,21.02,santo andre,SP,2685.00,895.000000,895.00,895.00,895.000
1,001cca7ae9ae17fb1caed9dfb1094831,12.488725,ffe4972089425f7752aad086a7fdce28,72.000000,006e43460a55bc60c0a437521e426529,3.000000,37.577164,ffe4972089425f7752aad086a7fdce28,114.62,006e43460a55bc60c0a437521e426529,14.72,cariacica,ES,30612.42,128.085439,499.50,69.90,99.990
2,001e6ad469a905060d959994f1b41e4f,11.548997,8501926dd0837d694fc5af339c02a6b2,11.548997,8501926dd0837d694fc5af339c02a6b2,11.548997,17.940000,8501926dd0837d694fc5af339c02a6b2,17.94,8501926dd0837d694fc5af339c02a6b2,17.94,sao goncalo,RJ,250.00,250.000000,250.00,250.00,250.000
3,002100f778ceb8431b7a1020ff7ab48f,14.814686,f4fdb235ce7c1f7d3eb1b2fdd3a4f610,40.000000,044cd7ffb6a41e6e5513c6595a2893ca,6.000000,14.505098,f4fdb235ce7c1f7d3eb1b2fdd3a4f610,34.15,044cd7ffb6a41e6e5513c6595a2893ca,4.91,franca,SP,1318.00,23.963636,129.90,9.90,17.900
4,003554e2dce176b5555353e4f3555ac8,4.000000,7e4f454abfc163899a6ef5b4d5facfb2,4.000000,7e4f454abfc163899a6ef5b4d5facfb2,4.000000,19.380000,7e4f454abfc163899a6ef5b4d5facfb2,19.38,7e4f454abfc163899a6ef5b4d5facfb2,19.38,goiania,GO,120.00,120.000000,120.00,120.00,120.000
5,004c9cd9d87a3c30c522c48c4fc07416,13.915487,ff9424310808d51de01b5e582c829336,66.000000,044223df7cb37509bfeb2a0df1250ca4,4.000000,21.025839,ff9424310808d51de01b5e582c829336,133.73,044223df7cb37509bfeb2a0df1250ca4,7.67,ibitinga,SP,21896.38,128.802235,779.97,47.90,113.995
6,00720abe85ba0859807595bbf045a33b,11.076923,d9a4f0796fafbfe369d8156189581cef,38.000000,002c9def9c9b951b1bec6d50753c9891,2.000000,13.460000,d9a4f0796fafbfe369d8156189581cef,38.60,002c9def9c9b951b1bec6d50753c9891,6.10,guarulhos,SP,1555.90,59.842308,156.00,13.50,51.800
7,00ab3eff1b5192e5f1a63bcecfee11c8,9.000000,deef94d95b5cf9c23e10621ad71a96b2,9.000000,deef94d95b5cf9c23e10621ad71a96b2,9.000000,12.080000,deef94d95b5cf9c23e10621ad71a96b2,12.08,deef94d95b5cf9c23e10621ad71a96b2,12.08,sao paulo,SP,98.00,98.000000,98.00,98.00,98.000
8,00d8b143d12632bad99c0ad66ad52825,7.000000,1e13bca551a3fd095b40e24798569805,7.000000,1e13bca551a3fd095b40e24798569805,7.000000,51.100000,1e13bca551a3fd095b40e24798569805,51.10,1e13bca551a3fd095b40e24798569805,51.10,belo horizonte,MG,86.00,86.000000,86.00,86.00,86.000
9,00ee68308b45bc5e2660cd833c3f81cc,8.785185,ff69de2237431040b7e8d1fcd79c0a7a,66.000000,03681e41ebddb2ca0241b148a40773d0,1.000000,19.836029,ff69de2237431040b7e8d1fcd79c0a7a,98.02,03681e41ebddb2ca0241b148a40773d0,2.34,sao paulo,SP,26280.00,152.790698,508.00,48.00,121.500


In [48]:
delivery_sales.columns

Index(['seller_id', 'mean_shipping_time', 'order_id_x', 'max_shipping_time',
       'order_id_y', 'min_shipping_time', 'mean_freight_value', 'order_id_x',
       'max_freight_value', 'order_id_y', 'min_freight_value', 'seller_city',
       'seller_state', 'items_value', 'mean_sales_value', 'max_sales_value',
       'min_sales_value', 'median_sales_value'],
      dtype='object')